In [ ]:
print(("Let's get this money!!"))

Let's get this money!!


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#!cp /content/gdrive/MyDrive/HW5_train_val.zip /content
#!unzip HW5_train_val.zip
#!cp /content/gdrive/MyDrive/HW5_test.zip /content
!mkdir /content/test/

!mv HW5_test.zip /content/test/  

!unzip /content/test/HW5_test.zip -d /content/test/

In [ ]:
import numpy as np
import torch
import torchvision   
import torch.nn as nn
from torchvision import transforms

import torch.nn.functional as F
import torch.utils.data as data
from PIL import Image
from torch.utils.data import Dataset, DataLoader
cuda = torch.device('cuda')
cpu = torch.device('cpu')
device = cuda

BATCH_SIZE = 16

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        upsample_block_num = 2

        super(Generator, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=9, padding=4),
            nn.PReLU()
        )
        self.block2 = ResidualBlock(64)
        self.block3 = ResidualBlock(64)
        self.block4 = ResidualBlock(64)
        self.block5 = ResidualBlock(64)
        self.block6 = ResidualBlock(64)
        self.block7 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64)
        )
        block8 = [UpsampleBLock(64, 2) for _ in range(upsample_block_num)]
        block8.append(nn.Conv2d(64, 3, kernel_size=9, padding=4))
        self.block8 = nn.Sequential(*block8)

    def forward(self, x):
        block1 = self.block1(x)
        block2 = self.block2(block1)
        block3 = self.block3(block2)
        block4 = self.block4(block3)
        block5 = self.block5(block4)
        block6 = self.block6(block5)
        block7 = self.block7(block6)
        block8 = self.block8(block1 + block7)

        return (torch.tanh(block8) + 1) / 2


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(512, 1024, kernel_size=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(1024, 1, kernel_size=1)
        )

    def forward(self, x):
        batch_size = x.size(0)
        return torch.sigmoid(self.net(x).view(batch_size))


class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x):
        residual = self.conv1(x)
        residual = self.bn1(residual)
        residual = self.prelu(residual)
        residual = self.conv2(residual)
        residual = self.bn2(residual)

        return x + residual


class UpsampleBLock(nn.Module):
    def __init__(self, in_channels, up_scale):
        super(UpsampleBLock, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels * up_scale ** 2, kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(up_scale)
        self.prelu = nn.PReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.pixel_shuffle(x)
        x = self.prelu(x)
        return x

#generator = Generator()
generator = torch.load("/content/gdrive/MyDrive/gen_v3.pth")
#discriminator = Discriminator()

In [ ]:
transform_train = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

# datasets = torchvision.datasets.ImageFolder(root='/home/ubuntu/content/DIV2K_valid_LR_bicubic')
# train_dataset_LR = torchvision.datasets.ImageFolder(root='/content/train_LR/', 
#                                                   transform=transform_train)
# dev_dataset_HR = torchvision.datasets.ImageFolder(root='/content/DIV2K_valid_HR/')
# train_dataset_HR = torchvision.datasets.ImageFolder(root='/content/train_HR/', 
#                                                    transform=transform_train)


In [ ]:
transform_train = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])
test_dataset = torchvision.datasets.ImageFolder(root='/content/test/', 
                                                   transform=transform_train)
test_loader = torch.utils.data.DataLoader(test_dataset)

In [ ]:
'/'.join(['submission']+test_dataset.imgs[0][0].split('/')[-2:])

'submission/comics/ARMS.png'

In [ ]:
transforms.ToPILImage()(generator(next(iter(test_dataset))[0].unsqueeze(0).cuda())[0]).save('/'.join(['submission']+test_dataset.imgs[0][0].split('/')[-2:]),format="png")

In [ ]:
!mkdir /content/submission/
!mkdir /content/submission/comics/
!mkdir /content/submission/large_test
!mkdir /content/submission/small_test/
!mkdir /content/submission/structures/

In [ ]:
for i, (img,_) in enumerate(test_dataset):
  filename = test_dataset.imgs[i][0]
  sr = generator(img.unsqueeze(0).cuda())[0]
  sr = transforms.ToPILImage()(sr)
  sr.save('/'.join(['submission']+filename.split('/')[-2:]), format="png")
  print(filename)

In [ ]:
!zip -r submission.zip submission

In [ ]:
!cp submission.zip /content/gdrive/MyDrive/